In [1]:
import sys, os
import matplotlib
matplotlib.use('Agg')
import numpy as np
import pandas as pd
import csv
import random
# import igraph
# import graph_tool as gt
import networkx as nx
# import utils.network as sn
# import matplotlib.pylab as plt
from scipy import stats
from sklearn.cluster import SpectralClustering
from sklearn import preprocessing
from sklearn import metrics
from graph_tool.all import *
import argparse

In [7]:
# sys.path.append('utils/')
# from preprocess import *
import os
import numpy as np
from numpy.core.numeric import identity
import pandas as pd
from scipy import stats
from statsmodels.stats.multitest import fdrcorrection
# internal lib
import network as sn

In [25]:
# inpath = '../GSE49710_original/'
# dataset = 'SEQC'

# inpath = '../TARGET_RNAseq/HTSeq_counts/'
inpath = '../TARGET_Methylation/'
dataset = 'TARGET'

In [26]:
if dataset == 'SEQC':
    array_dataset = pd.read_csv(inpath+"ArrayData_clean.tsv", sep='\t', index_col = 0)
    clinical_dataset = pd.read_csv(inpath+"clinical_data.tsv", sep='\t', index_col = 0)
    X = array_dataset.values
    Y = clinical_dataset.iloc[:, 1].values
elif dataset == 'TARGET':
    array_dataset = pd.read_csv(inpath + 'DataMat.csv', index_col = 0)
    clinical_dataset = pd.read_csv(inpath + 'ClinicalData.csv', index_col = 0)
    # print(clinical_dataset.head())
    X = array_dataset.values
    Y = clinical_dataset.values.reshape((-1,))
    # print(Y)
else: 
    print('unrecognized dataset type!')
    exit()

print(X.shape)
print(Y.shape)

(396065, 202)
(202,)


In [27]:

outcome_dict = dict()
row = 0

for i in array_dataset.columns:  # columns are patient samples
	row += 1
	outcome_dict[i] = Y[row-1]

print("outcome_dict:")
print(outcome_dict)

l = list(Y)
print(l)

outcome_dict:
{'F62': 1, 'F157': 1, 'F85': 1, 'F175': 0, 'F89': 1, 'F153': 0, 'F38': 1, 'F5': 1, 'F4': 1, 'F52': 0, 'F177': 0, 'F194': 1, 'F31': 0, 'F122': 0, 'F82': 0, 'F68': 1, 'F35': 1, 'F173': 0, 'F98': 0, 'F7': 1, 'F97': 0, 'F14': 1, 'F133': 1, 'F172': 0, 'F141': 1, 'F56': 1, 'F28': 0, 'F148': 1, 'F70': 1, 'F1': 1, 'F128': 1, 'F107': 0, 'F134': 0, 'F188': 1, 'F75': 1, 'F168': 1, 'F170': 1, 'F2': 1, 'F193': 1, 'F145': 1, 'F30': 0, 'F25': 0, 'F17': 1, 'F95': 0, 'F87': 1, 'F86': 1, 'F19': 0, 'F114': 0, 'F88': 1, 'F21': 1, 'F131': 0, 'F143': 0, 'F196': 0, 'F26': 0, 'F44': 1, 'F46': 0, 'F49': 1, 'F69': 0, 'F132': 1, 'F120': 1, 'F158': 0, 'F8': 0, 'F71': 1, 'F18': 0, 'F83': 0, 'F108': 0, 'F23': 0, 'F180': 1, 'F119': 1, 'F179': 0, 'F151': 1, 'F78': 1, 'F100': 1, 'F104': 0, 'F129': 1, 'F101': 0, 'F77': 1, 'F147': 0, 'F57': 1, 'F55': 0, 'F137': 1, 'F27': 1, 'F50': 0, 'F3': 1, 'F73': 1, 'F109': 0, 'F152': 1, 'F90': 0, 'F43': 0, 'F185': 0, 'F34': 0, 'F99': 1, 'F164': 0, 'F32': 1, 'F96': 1, '

In [40]:
def wilcoxon_test(x, label_list):
    # data = arr[:, :-1].reshape(-1, arr.shape[0])
    # label = arr[:, -1]
    # print(data.shape)
    # print(label.shape)

    p_values = []
    for i in range(len(x)):
        p = stats.wilcoxon(x[i], label_list).pvalue
        # rej_status, corrected_p = fdrcorrection(p, alpha=0.05)
        p_values.append(p)
        # p_values.append(corrected_p, i])
    
    ori_p = np.array(p_values)
    ori_keep_num = (ori_p < 0.001).sum()
    print('number of original p values < 0.001:', ori_keep_num)
    print('p values: ', ori_p)
    
    rej_status, corrected_p = fdrcorrection(p_values, alpha=0.001)
    # print(rej_status)
    # rej_num = len(np.where(rej_status==False))
    # print('number of rejected features:', rej_num)
    rej_indices = np.where(corrected_p >= 0.001)
    print('number of rejected features:', len(rej_indices))
    
#     keep_num = np.size(np.where(corrected_p < 0.1))
    keep_num = (corrected_p < 0.001).sum()
    print('number of preserved features:', keep_num)
    print('corrected p values:', corrected_p)
    return rej_status, rej_indices

In [41]:
_, unwanted_feature_indices = wilcoxon_test(X, l)
proc_X = np.delete(X, unwanted_feature_indices, axis=0)
print('feature number after wilcoxon:', len(proc_X))

number of original p values < 0.0001: 342887
p values:  [6.03954170e-01 8.59068689e-09 8.46213654e-11 ... 5.55684122e-02
 2.09685306e-01 1.02231924e-06]
number of rejected features: 1
number of preserved features: 352246
corrected p values: [6.13572981e-01 1.25408957e-08 5.25418686e-10 ... 5.91077273e-02
 2.18074282e-01 1.25985914e-06]
feature number after wilcoxon: 352246


In [42]:
proc_X.shape

(352246, 202)

In [43]:
# sum(unwanted_feature_indices)
print(unwanted_feature_indices)

(array([     0,      4,     14, ..., 396061, 396062, 396063]),)


In [47]:
graph = pd.read_csv(inpath+'new_adjmat.csv', index_col=0).values
graph.shape

(202, 202)

In [48]:
graph

array([[0.        , 0.2860023 , 0.31260367, ..., 0.23621613, 0.27119335,
        0.29248193],
       [0.2860023 , 0.        , 0.37556514, ..., 0.23501403, 0.28076552,
        0.2513116 ],
       [0.31260367, 0.37556514, 0.        , ..., 0.28433148, 0.25507711,
        0.25631968],
       ...,
       [0.23621613, 0.23501403, 0.28433148, ..., 0.        , 0.2522527 ,
        0.29811253],
       [0.27119335, 0.28076552, 0.25507711, ..., 0.2522527 , 0.        ,
        0.45919272],
       [0.29248193, 0.2513116 , 0.25631968, ..., 0.29811253, 0.45919272,
        0.        ]])